### Operations
- deleted hashtags ( not always good)
- used tweet-preprocessor removes
    - URLs   
    - Hashtags
    - Mentions
    - Reserved words (RT, FAV)
    - Emojis
    - Smileys
- removed numbers
- converted to lower

### Questions
- should you fill in Abbreviations (btw, imho ..)
- Should you use Lemmatization?
- should you use Stop Words?


In [2]:
import typing as tp
import numpy as np
import pandas as pd
import os

from collections import defaultdict
from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import re
#device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

pd.options.display.max_colwidth = 150

## Load Data

In [3]:
data = pd.read_csv('/home/iris/paulshab/Mynalabs_NLP/data/data_elonmusk.csv', encoding='windows-1252')
df = data
data.head()

,row ID,Tweet,Time,Retweet from,User
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",2017-09-29 17:39:19,NaN,elonmusk
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?",2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd,2017-09-29 09:19:21,NaN,elonmusk


## Hashtag Extraction using Regex

In [141]:
df['hashtag'] = df['Tweet'].apply(lambda x: re.findall(r"#(\w+)", x))
len(data)

3218

In [150]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.MENTION)

In [163]:
kk = " he had 5 children at #store in LA, @user123 tell me what this means. https://t.co/dGYDdGttYd "
print(p.clean(kk))
l = p.tokenize(kk)
print(l)

parsed_tweet = p.parse(kk)
print(parsed_tweet.hashtags)

he had 5 children at in LA, tell me what this means.
he had 5 children at in LA, $MENTION$ tell me what this means. $URL$
None


In [178]:
df = df.drop(1)

In [179]:
df.head(3)

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",2017-09-29 17:39:19,NaN,elonmusk,"Assuming max acceleration of to g's, but in a comfortable direction. Will feel like a mild to moder?"
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk,Yup
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk,Part


## Tweet Processing Libaray

In [ ]:
# #installing tweet-preprocessor
!pip install tweet-preprocessor

In [42]:
from ekphrasis.classes.segmenter import Segmenter
import preprocessor as p
#segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [184]:
len(df)

3218

In [185]:
#forming a separate feature for cleaned tweets
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.MENTION)

for i,v in enumerate(df['Tweet']):  # i --> num of tweet, v = tweet
    parsed_tweet = p.parse(v)
    if parsed_tweet.hashtags != None:
        df = df.drop(i)
    else:
        df.loc[i,'Cleaned_Tweet'] = p.clean(v)

len(df)

3116

In [154]:
df.head(5)

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",2017-09-29 17:39:19,NaN,elonmusk,"Assuming max acceleration of to g's, but in a comfortable direction. Will feel like a mild to moder?"
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?",2017-09-29 10:44:54,SpaceX,elonmusk,": BFR is capable of transporting satellites to orbit, crew and cargo to the and completing missions to the Moon an?"
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk,Yup
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk,Part
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd,2017-09-29 09:19:21,NaN,elonmusk,Fly to most places on Earth in under mins and anywhere in under . Cost per seat should be?


In [148]:
df.sample(5)

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet
1919,Row1919,Congrats to DeepMind! Many experts in the field thought AI was 10 years away from achieving this. https://t.co/5gGZZkud3K,2016-03-09 18:57:53,NaN,elonmusk,Congrats to DeepMind! Many experts in the field thought AI was years away from achieving this.
830,Row830,"@shails Yes, but shipping, taxes/tariffs and installation labor vary by country, as those costs are beyond our control",2017-03-10 09:27:06,NaN,elonmusk,"Yes, but shipping, taxes/tariffs and installation labor vary by country, as those costs are beyond our control"
1267,Row1267,"RT @TeslaMotors: Tesla and Panasonic to collaborate on photovoltaic cell and module production in Buffalo, New York https://t.co/bcDS3voHwu",2016-10-17 07:45:42,Tesla,elonmusk,": Tesla and Panasonic to collaborate on photovoltaic cell and module production in Buffalo, New York"
2714,Row2714,"Abort was caused by oxygen in ground side TEA-TEB. Upper stage on separate internal circuit, so doesn't face same risk.",2013-11-30 17:28:37,NaN,elonmusk,"Abort was caused by oxygen in ground side TEA-TEB. Upper stage on separate internal circuit, so doesn't face same risk."
2810,Row2810,UDP (yeah you know me) 3X,2013-07-09 23:54:48,NaN,elonmusk,UDP (yeah you know me) X


## Tokenization , Removal of Digits, Stop Words and Punctuations


In [110]:
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer


[nltk_data] Downloading package wordnet to /home/iris/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /home/iris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [116]:
# Remove Digits and lower the text
df['Cleaned_Tweet'] = df['Cleaned_Tweet'].astype(str).str.replace('\d+', '')
df['Cleaned_Tweet'] = df['Cleaned_Tweet'].str.lower()

/home/iris/paulshab/NLP_env/nlp_environ/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [173]:
df.sample(15)

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet
2864,Row2864,"Climate change deniers claim scientists disagree, same rebuttal used by tobacco industry about lung cancer for decades",2013-05-25 21:08:54,NaN,elonmusk,"Climate change deniers claim scientists disagree, same rebuttal used by tobacco industry about lung cancer for decades"
313,Row313,"@newscientist The world's population is accelerating towards collapse, but few seem to notice or care",2017-07-06 15:29:56,NaN,elonmusk,"The world's population is accelerating towards collapse, but few seem to notice or care"
3108,Row3108,RT @willsamari: @elonmusk you should let everyone know they can watch the launch live at: http://t.co/89qFtbZj74,2013-03-01 17:01:43,willsamari,elonmusk,: you should let everyone know they can watch the launch live at:
1171,Row1171,RT @TeslaMotors: Survey shows Tesla owners in Germany understand the meaning and function of Autopilot https://t.co/gvEX0h7mxq,2016-11-10 20:56:14,Tesla,elonmusk,: Survey shows Tesla owners in Germany understand the meaning and function of Autopilot
2759,Row2759,RT @ID_AA_Carmack: @elonmusk @BadAstronomer We actually are outside some of the IPCC prediction ranges from 20 years ago http://t.co/WOLZlZ?,2013-09-27 02:35:33,ID_AA_Carmack,elonmusk,: We actually are outside some of the IPCC prediction ranges from years ago ?
738,Row738,@business Glad to have Tencent as an investor and advisor to Tesla,2017-03-28 20:04:59,NaN,elonmusk,Glad to have Tencent as an investor and advisor to Tesla
3063,Row3063,@Nezheim @jbmeicher sure :),2013-03-24 01:41:56,NaN,elonmusk,sure
16,Row16,"@El_Deano @Daimler @Jalopnik Yes, I did :)",2017-09-26 15:04:25,NaN,elonmusk,"Yes, I did"
144,Row144,"@neddesmond @SpaceX The danger is def high for South Korea, due to proximity, but much less for distant targets lik? https://t.co/H6w2W9Mage",2017-08-15 01:26:47,NaN,elonmusk,"The danger is def high for South Korea, due to proximity, but much less for distant targets lik?"
1674,Row1674,"@jlgolson No prob. We care about doing what's fair & just. Don't always get it right, but intent is always there & we make amends when wrong",2016-05-18 22:47:37,NaN,elonmusk,"No prob. We care about doing what's fair & just. Don't always get it right, but intent is always there & we make amends when wrong"


In [127]:
df.iloc[[3211]]['Cleaned_Tweet']

3211    btw, i don't think apple is doomed. just won't...
Name: Cleaned_Tweet, dtype: object

In [180]:
df.iloc[1,5]

'Yup'

In [181]:
df.iloc[1,1]

'@bigajm Yup :)'

In [188]:
import preprocessor

In [ ]:
#climate change is bad

In [186]:
new_data = pd.read_csv('/home/iris/paulshab/Mynalabs_NLP/Twitter_Processing/file2.csv', encoding='windows-1252')
new_data 

,row ID,Tweet,Time,Retweet from,User,Cleaned_Tweet
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",2017-09-29 17:39:19,NaN,elonmusk,"Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder?"
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?",2017-09-29 10:44:54,SpaceX,elonmusk,": BFR is capable of transporting satellites to orbit, crew and cargo to the and completing missions to the Moon an?"
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk,Yup
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk,Part 2
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd,2017-09-29 09:19:21,NaN,elonmusk,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be?
...,...,...,...,...,...,...
3111,Row3212,"Mass. judge denies auto dealers' demand to kill our little Tesla store. Yay, justice prevails! http://t.co/MqnNYScE",2012-11-20 21:22:29,NaN,elonmusk,"Mass. judge denies auto dealers' demand to kill our little Tesla store. Yay, justice prevails!"
3112,Row3213,"@YOUSRC Amos's article was fair, but his editor chose a more sensational headline & used quotes even though I didn't actually say that.",2012-11-20 08:52:03,NaN,elonmusk,"Amos's article was fair, but his editor chose a more sensational headline & used quotes even though I didn't actually say that."
3113,Row3214,These articles in Space News describe why Ariane 6 vs 5: http://t.co/IaeYXXK9 and http://t.co/ghs3FG8w,2012-11-20 08:38:31,NaN,elonmusk,These articles in Space News describe why Ariane 6 vs 5: and
3114,Row3215,"Was misquoted by BBC as saying Europe's rocket has no chance. Just said the [Franco-German] Ariane 5 has no chance, so go with Ariane 6.",2012-11-20 08:30:44,NaN,elonmusk,"Was misquoted by BBC as saying Europe's rocket has no chance. Just said the [Franco-German] Ariane 5 has no chance, so go with Ariane 6."


In [189]:
import pandas

In [195]:
def install(module, package, short_cut=None):
    import importlib
    try:
        importlib.import_module(module)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        if short_cut != None:
            globals()[module] = importlib.import_module(module)
        else:
            import module

install(module='pandas', package='pandas', short_cut='pd')

SyntaxError: invalid syntax (3434669239.py, line 10)

In [201]:
import importlib
importlib.import_module??

In [204]:
import subprocess
import sys

def install(module, package, short_cut=None):
    print()
    try:
        if short_cut != None:
            import module as short_cut
        else:
            import module
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        if short_cut != None:
            print("hello")
            print(module)
            import module as short_cut
        else:
            import module
    
install(module='pandas', package='pandas', short_cut='pd')

hello
pandas


You should consider upgrading via the '/home/iris/paulshab/NLP_env/nlp_environ/bin/python -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'module'

In [205]:
subprocess.call(['pip', 'install', 'wget'])

  Using cached wget-3.2-py3-none-any.whl


You should consider upgrading via the '/home/iris/paulshab/NLP_env/nlp_environ/bin/python -m pip install --upgrade pip' command.


0

In [9]:
data.head()

,row ID,Tweet,Time,Retweet from,User
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",2017-09-29 17:39:19,NaN,elonmusk
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?",2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd,2017-09-29 09:19:21,NaN,elonmusk


In [12]:
data = pd.read_csv('/home/iris/paulshab/Mynalabs_NLP/data/data_elonmusk.csv', encoding='windows-1252')
df = data
data = data.drop(['Time', 'Retweet from', 'User'], axis=1)
data.head()

,row ID,Tweet
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC"
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?"
2,Row2,@bigajm Yup :)
3,Row3,Part 2 https://t.co/8Fvu57muhM
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd


In [24]:
import nltk 
import wget
import preprocessor as p

In [25]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.MENTION)
url = 'https://raw.githubusercontent.com/camaro1200/Tweet_Processing/main/data_elonmusk.csv'

def clean_mode1(df, min_len):
    """
        - deltes all tweets with hastags
        - removes hashtags/mentions/reserved words/Emojis/Smileys
    """
    
    for i,v in enumerate(df['Tweet']):
        parsed_tweet = p.parse(v)
        if parsed_tweet.hashtags != None:
            df = df.drop(i)
        else:
            cleaned_tweet = p.clean(v)
            tokens = nltk.word_tokenize(cleaned_tweet)
            if len(tokens) < min_len:
                df = df.drop(i)
            else:
                df.loc[i,'Cleaned_Tweet'] = cleaned_tweet
        
     
    return df


class CreateDataset():
    def __init__(self, mode, ltweet, ofile):
        self.ltweet = ltweet
        self.ifile = wget.download(url)
        self.mode = mode
        self.ofile = ofile
    
    def load_dataset(self):
        
        df = pd.read_csv(self.ifile, encoding='windows-1252')
        df = df.drop(['Time', 'Retweet from', 'User'], axis=1)
        
        if self.mode == 0:
            print("no changes made")
        elif self.mode == 1:
            df = clean_mode1(df, self.ltweet)
        else:
            print ("no such mode found, please enter valid mode")
            sys.exit(2)
        
        #df.to_csv(self.ofile, index=False)
        
        return df
        

In [28]:
dataset = CreateDataset(mode=1, ltweet=3, ofile="exit.txt")
new_df = dataset.load_dataset()

100% [............................................................................] 452905 / 452905

In [29]:
new_df.head()

,row ID,Tweet,Cleaned_Tweet
0,Row0,"@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC","Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder?"
1,Row1,"RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?",": BFR is capable of transporting satellites to orbit, crew and cargo to the and completing missions to the Moon an?"
4,Row4,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd,Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be?
5,Row5,"RT @SpaceX: Supporting the creation of a permanent, self-sustaining human presence on Mars. https://t.co/kCtBLPbSg8 https://t.co/ra6hKsrOcG",": Supporting the creation of a permanent, self-sustaining human presence on Mars."
6,Row6,BFR will take you anywhere on Earth in less than 60 mins https://t.co/HWt9BZ1FI9,BFR will take you anywhere on Earth in less than 60 mins


In [16]:
tokens = nltk.word_tokenize("hello how are you")
tokens

['hello', 'how', 'are', 'you']